# Weight of Evidence on dataset
<br>
<br>
Weight of Evidence - Variable importance technique

##### Import dataset: Using ... from Wooldridge

In [7]:
# Invite all of our friends to the party
import pandas as pd
import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

In [8]:
df = pd.read_csv('/home/justin/jdwall1/wooldridge/raw_data/data_csv/pntsprd.csv')

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553 entries, 0 to 552
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   favscr   553 non-null    int64  
 1   undscr   553 non-null    int64  
 2   spread   553 non-null    float64
 3   favhome  553 non-null    int64  
 4   neutral  553 non-null    int64  
 5   fav25    553 non-null    int64  
 6   und25    553 non-null    int64  
 7   fregion  553 non-null    int64  
 8   uregion  553 non-null    int64  
 9   scrdiff  553 non-null    int64  
 10  sprdcvr  553 non-null    int64  
 11  favwin   553 non-null    int64  
dtypes: float64(1), int64(11)
memory usage: 52.0 KB


In [9]:
df.head()

,favscr,undscr,spread,favhome,neutral,fav25,und25,fregion,uregion,scrdiff,sprdcvr,favwin
0,72,61,7.0,0,0,1,0,3,4,11,1,1
1,82,74,7.0,1,0,0,0,3,1,8,1,1
2,87,57,17.0,1,0,0,0,3,3,30,1,1
3,69,70,9.0,1,0,0,0,3,3,-1,0,0
4,77,79,2.5,0,0,0,0,2,3,-2,0,0


In [10]:
# Rename the dependent variable to 'target'
df1=df.rename(columns={'favwin':'target'})

In [11]:
df1.head()

,favscr,undscr,spread,favhome,neutral,fav25,und25,fregion,uregion,scrdiff,sprdcvr,target
0,72,61,7.0,0,0,1,0,3,4,11,1,1
1,82,74,7.0,1,0,0,0,3,1,8,1,1
2,87,57,17.0,1,0,0,0,3,3,30,1,1
3,69,70,9.0,1,0,0,0,3,3,-1,0,0
4,77,79,2.5,0,0,0,0,2,3,-2,0,0


In [27]:
#This is the WOE function. Peruse at your leisure

max_bin = 20
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

# Define function for character binning
def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

# Define final function with dataframe and target
def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [14]:
# Run the data_vars function and return the IV and the
final_iv, IV = data_vars(df1,df1.target)

In [29]:
# Final information value output - all the details
final_iv.head()

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,favscr,43.0,65.0,93,38,0.408602,55,0.591398,0.090047,0.419847,-1.539555,0.806461
1,favscr,66.0,72.0,105,69,0.657143,36,0.342857,0.163507,0.274809,-0.519220,0.806461
2,favscr,73.0,77.0,84,64,0.761905,20,0.238095,0.151659,0.152672,-0.006657,0.806461
3,favscr,78.0,83.0,88,76,0.863636,12,0.136364,0.180095,0.091603,0.676019,0.806461
4,favscr,84.0,91.0,96,88,0.916667,8,0.083333,0.208531,0.061069,1.228087,0.806461


In [15]:
# The variable name and information value associated from WOE above
IV.sort_values('IV',ascending=False)

,VAR_NAME,IV
7,spread,0.991844
2,favscr,0.806461
6,sprdcvr,0.759791
9,undscr,0.751937
5,scrdiff,0.694185
1,favhome,0.096495
8,und25,0.052197
4,neutral,0.039628
3,fregion,0.012188
0,fav25,0.006789


In [30]:
# Some say to use variables between 0.10 and 0.50 information value
df_woe=IV.query('(IV > 0.10)')

In [31]:
# Create list of varible names from filter above
df_iv=df_woe['VAR_NAME']

In [32]:
# Create new dataframe from original dataframe and subset of variables from list above
df1a=df1[df_iv]

In [35]:
# Merge back the new df to the target
df2 = pd.merge(df1a, df1['target'],left_index=True,right_index=True)

In [36]:
# New set of variables seen here
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553 entries, 0 to 552
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   favscr   553 non-null    int64  
 1   scrdiff  553 non-null    int64  
 2   sprdcvr  553 non-null    int64  
 3   spread   553 non-null    float64
 4   undscr   553 non-null    int64  
 5   target   553 non-null    int64  
dtypes: float64(1), int64(5)
memory usage: 26.0 KB
